In [1]:
import pandas as pd
import logging
import tqdm
import time
from pathlib import Path

import torch
from transformers import BartForSequenceClassification, BartTokenizer, BartConfig
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# internal libraries
from resources import target_to_label

# set a seed value
torch.manual_seed(2022)

log = logging.getLogger(__name__)
log.setLevel(logging.WARNING)

/Users/guillaume/hackathon-odd/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Params**

In [2]:
results_dir = Path("results")

"""Check true target in X highest predicted targets"""
top_X_targets = 5

model_path = "valhalla/distilbart-mnli-12-9"

dataset_path = "data/afd_targets_odd_12_15_16.csv"

In [3]:
targets = list(target_to_label.keys())
targets.append("0")

labels = list(target_to_label.values())

results_dir.mkdir(exist_ok=True)    # create output dir
now = time.time()

label_to_target = {v: k for k, v in target_to_label.items()}

# create output file
with open(results_dir / Path(f"zs-nli-{now}.csv"), "w") as f:
    f.write(
        "text,target1,target2,exec_time,"
        + ",".join([k for k in target_to_label.keys()])
        + "\n"
    )

In [4]:
config = BartConfig.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForSequenceClassification.from_pretrained(model_path)
model.eval()

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((10

In [5]:
def predict(premise, hypothesis):
    """
    Function to run a prediction
    :param premise:
    :param hypothesis: Hypothesis based on the premise
    :return: Probability that the hypothesis is true based on the premise
    """
    # run through model pre-trained on MNLI
    # :warning: text is truncate
    input_ids = tokenizer.encode(
        premise, hypothesis, truncation=True, return_tensors="pt"
    )
    with torch.no_grad():
        logits = model(input_ids)[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true
    entail_contradiction_logits = logits[:, [0, 2]]

    probs = entail_contradiction_logits.softmax(dim=1)
    true_prob = probs[:, 1].item() * 100
    logging.info(f"Probability that '{hypothesis}' is true: {true_prob:0.2f}%")

    return true_prob

In [6]:
# df true
df_true = pd.read_csv(dataset_path)

y_true, y_pred = [], []

pbar = tqdm.tqdm(150)

for index, row in df_true.iterrows():  # text

    results = {k: "" for k in target_to_label.keys()}

    start_time = time.time()

    text = row["text"]
    for label in labels:
        # Build hypothesis
        hypothesis = "The context is " + label

        # Run prediction
        true_prob = predict(text, hypothesis)

        target = label_to_target[label]
        results[target] = true_prob

    exec_time = time.time() - start_time
    logging.info(f"Total prediction time : {exec_time:0.2f}s")    

    # add result to output file
    with open(results_dir / Path(f"zs-nli-{now}.csv"), "a") as f:
        new_line = (
            f'"{text}",{exec_time},{row["target1"]},{row["target2"]},'
            + ",".join([f"{v:.2f}" for v in results.values()])
            + "\n"
        )
        f.write(new_line)

    top_X_targets_list = []
    target_highest_score = max(results, key=results.get)
    # find X targets with highest score
    for _ in range(top_X_targets):
        target_max_temp = max(results, key=results.get)

        top_X_targets_list.append(target_max_temp)
        results.pop(target_max_temp)

    y_true.append(row["target1"])
    # if truth in X highest targets, append truth
    if row["target1"] in top_X_targets_list:
        y_pred.append(row["target1"])
    # else append target with the highest score
    else:
        y_pred.append(target_highest_score)

    del results
    pbar.update(1)

65it [21:52, 20.19s/it]

In [ ]:
print(classification_report(y_true, y_pred, labels=targets))

fig, ax = plt.subplots(figsize=(20, 15))
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, labels=targets, ax=ax)